In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
!pip install transformers
!pip install sentencepiece
!pip install tqdm

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [2]:
import pandas as pd
import numpy as np
from transformers import BertTokenizer, BertModel
from tokenizers import BertWordPieceTokenizer
from nltk.corpus import stopwords
from nltk import pad_sequence
import os
import torch
from nltk import word_tokenize
import tqdm
import tensorflow as tf


ModuleNotFoundError: ignored

In [ ]:
import nltk
nltk.download('stopwords')
nltk.download('punkt')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!
[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Package punkt is already up-to-date!


True

In [ ]:
# data = pd.read_csv('/content/drive/MyDrive/labeled_benchmark/datalake/data_mill_a.csv')
# # data = pd.read_csv('/content/abandoned_wells_0.csv')
# data= data.astype('str')
# data.head(10)

In [ ]:
def tokenize(row):
#   maxi = 30
  tok_sw = []
  word_idx = {}
  temp = []
  stop_words = set(stopwords.words("english"))

  #split the values 
#   print(row)
  row_token = word_tokenize(row)
  # print(row_token)

  #remove stopwords
  for i in row_token:
    if i not in stop_words and i not in tok_sw:
      tok_sw.append(i)

  #find indices for the tokens
  for i in tok_sw:
      s = tok_sw.index(i)
      temp.append(s)
  temp.sort()
  for i in temp:
        word_idx[tok_sw[i]] = temp.index(i)+1

  tokenised_words = []
  index = []
  for key, val in word_idx.items():
    tokenised_words.append(key)
    index.append(val)

  return tokenised_words, index

In [ ]:
import random
def data_preprocessing(data):
  data_samples = pd.DataFrame()
  # dataset = pd.read_csv(data)
  data = data.dropna(axis=0)
  cols = data.columns.to_list()
  # print(len(data))
  # data = data.sample(30, replace=False)
  # print(len(data.values.tolist()))
  if len(data) > 20:
    data_samples = pd.DataFrame(random.sample(data.values.tolist(), k=20))
  elif len(data) > 0:
    data_samples = data
  
  print(len(data_samples))
  processed_data = []
  cols = []
  
  for i in range(len(data_samples)):
    row_str = " "
    for j in range(len(cols)):
      if type(data_samples.iloc[i][j]) != None:
        row_str += str(data_samples.iloc[i][j])+' '
    processed_data.append(row_str)
  return processed_data

In [ ]:
# data = "/content/drive/MyDrive/labeled_benchmark/datalake/data_mill_0.csv"
# p = data_preprocessing(data)
# # print(p)

In [ ]:
from tokenizers import BertWordPieceTokenizer
import os
def token_training(processed_data):
  with open("processed.txt", 'a') as f:
    for i in processed_data:
      f.write(i)
  path = "processed.txt"
  # initialize
  tokenizer = BertWordPieceTokenizer(
      clean_text=True,
      handle_chinese_chars=False,
      strip_accents=False,
      lowercase=False
  )
  # and train
  tokenizer.train(files=path, vocab_size=30_000, min_frequency=0,
                  limit_alphabet=1000, wordpieces_prefix='##',
                  special_tokens=['[PAD]', '[CLS]', '[SEP]', '[MASK]', '[UNK]'])

  # os.mkdir('./bert-custom')
  tokenizer.save_model('./')

In [ ]:
def process(processed_data):
  """
    read text file
    divide into batches
    read each sentence
    tokenise, pad, [SEP]
    bert
  """
  model = BertModel.from_pretrained('bert-base-uncased', output_hidden_states = True,)
  # with open(text_file) as f:
  #   d = f.readlines() #returns a list of strings
  # do = []
  # for i in d:
  #   do.append(i.strip("\n "))
  
  ##### Reading text file done #####

  batch_str = []
  size = 1
  start = 0
  batches = [processed_data[i:i+size] for i in range(0,len(processed_data),size)]
  
  
  for batch in batches:
    sent_str = ""
    for b in batch:
      sent_str += b + ' '
    batch_str.append(sent_str)
  
  # ##### Batch division done #####

  tokens = []
  T = 512
  token_training(processed_data)
  tokenizer = BertTokenizer.from_pretrained('./vocab.txt')
  for row in tqdm.tqdm(batch_str):
    r = '[CLS]' + row + '[SEP]'
    tokens.append(tokenizer.tokenize(r)) # tokenize the sentences
  hidden_states = []

  for token in tokens:
    
    padded_tokens = token+ ['[PAD]' for _ in range(T-len(token))]
    

    attn_mask=[ 1 if token != '[PAD]' else 0 for token in padded_tokens ]
    seg_ids=[0 for _ in range(len(padded_tokens))]

    row_ids = [tokenizer.convert_tokens_to_ids(word) for word in padded_tokens]
    # print(row_ids)

    # print(None in row_ids)
    if None in row_ids:
      print(f'The word {padded_tokens[row_ids.index(None)]} was tokenized as None')
    row_ids = torch.tensor(row_ids).to(torch.int32)
    
    token_ids = torch.tensor(row_ids).unsqueeze(0)
    # print(token_ids.size())
    attn_mask = torch.tensor(attn_mask).unsqueeze(0)
    seg_ids = torch.tensor(seg_ids).unsqueeze(0)
    with torch.no_grad():
      model.eval()
      hidden_reps = model(input_ids=token_ids, attention_mask = attn_mask, token_type_ids = seg_ids)
      hidden_states.append(hidden_reps[2][-1])
  return hidden_states


In [ ]:
# p = data_preprocessing(data)

In [ ]:
# res = process(p)
# res

In [ ]:
def table_embedding(data):
  processed_data = data_preprocessing(data)
  pooler_op = process(processed_data)
  pooler_sum = torch.empty((1, 512, 768))
  for i in pooler_op:
    pooler_sum += i
  pooler_avg = pooler_sum / len(pooler_op)
  return pooler_avg


In [ ]:
!pip3 install pickle5
import pickle5 as pickle
with open("/content/drive/MyDrive/labeled_benchmark/santosUnionBenchmark (1).pickle", "rb") as fh:
  groundtruth = pickle.load(fh)

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/


In [ ]:
print(groundtruth)

{'data_mill_a.csv': ['data_mill_a.csv', 'data_mill_b.csv', 'data_mill_0.csv', 'data_mill_1.csv', 'data_mill_2.csv', 'data_mill_3.csv', 'data_mill_4.csv', 'data_mill_5.csv', 'data_mill_6.csv', 'data_mill_7.csv', 'data_mill_8.csv', 'data_mill_9.csv', 'data_mill_10.csv', 'data_mill_11.csv'], 'data_mill_b.csv': ['data_mill_a.csv', 'data_mill_b.csv', 'data_mill_0.csv', 'data_mill_1.csv', 'data_mill_2.csv', 'data_mill_3.csv', 'data_mill_4.csv', 'data_mill_5.csv', 'data_mill_6.csv', 'data_mill_7.csv', 'data_mill_8.csv', 'data_mill_9.csv', 'data_mill_10.csv', 'data_mill_11.csv'], 'cihr_co-applicant_a.csv': ['cihr_co-applicant_a.csv', 'cihr_co-applicant_b.csv', 'cihr_co-applicant_0.csv', 'cihr_co-applicant_1.csv', 'cihr_co-applicant_2.csv', 'cihr_co-applicant_3.csv', 'cihr_co-applicant_4.csv', 'cihr_co-applicant_5.csv', 'cihr_co-applicant_6.csv', 'cihr_co-applicant_7.csv', 'cihr_co-applicant_8.csv', 'cihr_co-applicant_9.csv'], 'cihr_co-applicant_b.csv': ['cihr_co-applicant_a.csv', 'cihr_co-appl

In [ ]:
pairs = {}
datalake = set()
query = set()
for key, value in groundtruth.items():
  query.add(key)
  for v in value:
    datalake.add(v)
    pairs[(key, v)] = '1'
res =dict(reversed(list(pairs.items())))

print(res)

{('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_9.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_8.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_7.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_6.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_5.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_4.csv'): '1', ('cdc_nutrition_physical_activity_and_obesity_legislation_a.csv', 'cdc_nutrition_physical_activity_and_obesity_legislation_3.csv'): '1', ('cdc_nutrition_physical_activity_and_ob

In [ ]:
Q_path = "/content/drive/MyDrive/labeled_benchmark/query/"
file_len = {}
for i in query:
  df = pd.read_csv(Q_path+i)
  file_len[i] = len(df)
sort_dict = dict(sorted(file_len.items(), key=lambda item: item[1]))
print(sort_dict)



{'workforce_management_information_a.csv': 17, 'minister_meetings_a.csv': 17, 'workforce_management_information_b.csv': 18, 'civic_building_locations_a.csv': 26, 'senior_officials_expenses_a.csv': 59, 'job_pay_scales_a.csv': 129, 'purchasing_card_a.csv': 257, 'practice_reference_a.csv': 321, 'practice_reference_b.csv': 325, 'stockport_contracts_a.csv': 358, 'stockport_contracts_b.csv': 501, 'ydn_spending_data_b.csv': 568, 'ydn_spending_data_a.csv': 722, 'albums_a.csv': 836, 'ipopayments_a.csv': 918, 'ipopayments_b.csv': 967, 'albums_b.csv': 969, 'film_locations_in_san_francisco_a.csv': 970, 'mines_b.csv': 1121, 'deaths_2012_2018_a.csv': 1138, 'time_spent_watching_vcr_movies_a.csv': 1206, 'abandoned_wells_a.csv': 1857, 'animal_tag_data_b.csv': 5000, 'contributors_parties_b.csv': 5000, 'complaint_by_practice_b.csv': 5000, 'lane_description_b.csv': 5088, 'monthly_data_feed_b.csv': 5376, 'lane_description_a.csv': 6000, 'business_rates_a.csv': 6307, 'mines_a.csv': 6442, 'monthly_data_feed_a

In [ ]:
# fil = ['minister_meetings_a.csv', 'workforce_management_information_a.csv', 'workforce_management_information_b.csv', 'civic_building_locations_a.csv', 'senior_officials_expenses_a.csv']
gt = list(pairs.keys())
# op = []
# for i in range(len(fil)):
#   for j in range(len(gt)):
#     if fil[i] in gt[j]:
#       op.append(gt[j])
# lst_of_tab = list(set(op))

print(gt)

[('data_mill_a.csv', 'data_mill_a.csv'), ('data_mill_a.csv', 'data_mill_b.csv'), ('data_mill_a.csv', 'data_mill_0.csv'), ('data_mill_a.csv', 'data_mill_1.csv'), ('data_mill_a.csv', 'data_mill_2.csv'), ('data_mill_a.csv', 'data_mill_3.csv'), ('data_mill_a.csv', 'data_mill_4.csv'), ('data_mill_a.csv', 'data_mill_5.csv'), ('data_mill_a.csv', 'data_mill_6.csv'), ('data_mill_a.csv', 'data_mill_7.csv'), ('data_mill_a.csv', 'data_mill_8.csv'), ('data_mill_a.csv', 'data_mill_9.csv'), ('data_mill_a.csv', 'data_mill_10.csv'), ('data_mill_a.csv', 'data_mill_11.csv'), ('data_mill_b.csv', 'data_mill_a.csv'), ('data_mill_b.csv', 'data_mill_b.csv'), ('data_mill_b.csv', 'data_mill_0.csv'), ('data_mill_b.csv', 'data_mill_1.csv'), ('data_mill_b.csv', 'data_mill_2.csv'), ('data_mill_b.csv', 'data_mill_3.csv'), ('data_mill_b.csv', 'data_mill_4.csv'), ('data_mill_b.csv', 'data_mill_5.csv'), ('data_mill_b.csv', 'data_mill_6.csv'), ('data_mill_b.csv', 'data_mill_7.csv'), ('data_mill_b.csv', 'data_mill_8.csv'

In [ ]:
print(f"query", len(query))
print(f"datalake", len(datalake))
print('abandoned_wells_0.csv' in query)

query 50
datalake 475
False


In [ ]:
import os
  
# # Folder Path for datalake
path = "/content/drive/MyDrive/labeled_benchmark/datalake"
  
# Change the directory
os.chdir(path)

def read_file(file_path):
    with open(file_path, 'r') as f:
      file_datalake = f.read()


In [ ]:
import csv
import pandas as pnd
datalake = list(datalake)
query = list(query)
emb = []
file_path_datalake = "/content/drive/MyDrive/labeled_benchmark/datalake/"
file_path_query = "/content/drive/MyDrive/labeled_benchmark/query/"


for x in gt[0:15]:
  if (x[1] in datalake) and (x[0] in query):
    df_dl = pnd.read_csv(file_path_datalake+x[1])
    df_q = pnd.read_csv(file_path_query+x[0])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb.append([dl_emb, q_emb])





20


Downloading:   0%|          | 0.00/570 [00:00<?, ?B/s]

Downloading:   0%|          | 0.00/440M [00:00<?, ?B/s]

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
/usr/local/lib/python3.8/dist-packages/transformers/tokenization_utils_base.py:1703: FutureWarning: Calling BertTokenizer.from_pretrained() with the path t

20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 22489.57it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 22531.85it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 7497.86it/s]


In [ ]:
for x in gt[15:35]:
  if (x[1] in datalake) and (x[0] in query):
    df_dl = pnd.read_csv(file_path_datalake+x[1])
    df_q = pnd.read_csv(file_path_query+x[0])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb.append([dl_emb, q_emb])


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 3985.65it/s]
<ipython-input-101-97c62983cf6f>:57: UserWarning: To copy construct from a tensor, it is recommended to u

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 17575.13it/s]


20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 24153.78it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 15316.06it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 23974.30it/s]


In [ ]:
for x in gt[35:50]:
  if (x[1] in datalake) and (x[0] in query):
    df_dl = pnd.read_csv(file_path_datalake+x[1])
    df_q = pnd.read_csv(file_path_query+x[0])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb.append([dl_emb, q_emb])


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 6851.20it/s]
<ipython-input-101-97c62983cf6f>:57: UserWarning: To copy construct from a tensor, it is recommended to u

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 14797.33it/s]


20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 11890.30it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 6984.69it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.transform.dense.weight', 'cls.predictions.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 24973.53it/s]


In [ ]:
torch.save(emb, '/content/drive/MyDrive/labeled_benchmark/embeddings.pt')

In [ ]:
arr_dl = os.listdir('/content/drive/MyDrive/labeled_benchmark/datalake')
arr_q = os.listdir('/content/drive/MyDrive/labeled_benchmark/query')

In [ ]:
zero_labels_datalake = []
zero_labels_query = []
res_pairs = []

    
for i, j in zip(arr_dl, arr_q):
  res_pairs.append((i, j))


In [ ]:
res_pairs

[('311_calls_historic_data_5.csv', '311_calls_historic_data_a.csv'),
 ('311_calls_historic_data_a.csv', 'albums_a.csv'),
 ('311_calls_historic_data_9.csv', 'abandoned_wells_a.csv'),
 ('311_calls_historic_data_4.csv', 'albums_b.csv'),
 ('311_calls_historic_data_3.csv', 'animal_tag_data_a.csv'),
 ('311_calls_historic_data_8.csv', 'animal_tag_data_b.csv'),
 ('311_calls_historic_data_7.csv',
  'cdc_nutrition_physical_activity_and_obesity_legislation_a.csv'),
 ('311_calls_historic_data_0.csv', 'complaint_by_practice_a.csv'),
 ('311_calls_historic_data_1.csv', 'complaint_by_practice_b.csv'),
 ('311_calls_historic_data_6.csv', 'contributors_parties_a.csv'),
 ('311_calls_historic_data_2.csv', 'contributors_parties_b.csv'),
 ('abandoned_wells_7.csv', 'deaths_2012_2018_a.csv'),
 ('albums_7.csv', 'film_locations_in_san_francisco_a.csv'),
 ('albums_8.csv', 'new_york_city_restaurant_inspection_results_a.csv'),
 ('abandoned_wells_6.csv',
  'oil_and_gas_summary_production_data_1967_1999_a.csv'),
 ('a

In [ ]:
# path_dl = '/content/drive/MyDrive/labeled_benchmark/datalake/'
# dl_file_len = {}
# for i in zero_labels_datalake[:-7]:
#   df = pd.read_csv(path_dl+i)
#   dl_file_len[i] = len(df)
# dl_sort_dict = dict(sorted(dl_file_len.items(), key=lambda item: item[1]))
# print(dl_sort_dict)

In [ ]:
emb_0 = []
file_path_datalake = "/content/drive/MyDrive/labeled_benchmark/datalake/"
file_path_query = "/content/drive/MyDrive/labeled_benchmark/query/"


for x in res_pairs[0:15]:
  if ((x[0] in arr_dl) and (x[1] in arr_q)):
    df_dl = pd.read_csv(file_path_datalake+x[0])
    df_q = pd.read_csv(file_path_query+x[1])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb_0.append([dl_emb, q_emb])

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 20530.12it/s]
<ipython-input-34-97c62983cf6f>:57: UserWarning: To copy construct from a tensor, it is recommended to u

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 7367.48it/s]


20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 4137.01it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 17196.82it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 24328.91it/s]


In [ ]:
for x in res_pairs[15:35]:
  if ((x[0] in arr_dl) and (x[1] in arr_q)):
    df_dl = pd.read_csv(file_path_datalake+x[0])
    df_q = pd.read_csv(file_path_query+x[1])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb_0.append([dl_emb, q_emb])

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 22604.71it/s]
<ipython-input-34-97c62983cf6f>:57: UserWarning: To copy construct from a tensor, it is recommended to u

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 21140.65it/s]


20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 10419.34it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 19636.25it/s]


2


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 2/2 [00:00<00:00, 7275.46it/s]


In [ ]:
for x in res_pairs[35:50]:
  if ((x[0] in arr_dl) and (x[1] in arr_q)):
    df_dl = pd.read_csv(file_path_datalake+x[0])
    df_q = pd.read_csv(file_path_query+x[1])
    dl_emb = table_embedding(df_dl)
    q_emb = table_embedding(df_q)
    emb_0.append([dl_emb, q_emb])

20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 7656.63it/s]
<ipython-input-34-97c62983cf6f>:57: UserWarning: To copy construct from a tensor, it is recommended to us

0


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
0it [00:00, ?it/s]


20


...

Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 22459.46it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 21076.90it/s]


20


Some weights of the model checkpoint at bert-base-uncased were not used when initializing BertModel: ['cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias', 'cls.predictions.transform.dense.bias', 'cls.predictions.decoder.weight', 'cls.predictions.transform.LayerNorm.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.seq_relationship.weight']
- This IS expected if you are initializing BertModel from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertModel from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
100%|██████████| 20/20 [00:00<00:00, 13327.94it/s]


In [ ]:
torch.save(emb_0, '/content/drive/MyDrive/labeled_benchmark/zero_embedding.pt')

In [2]:
import torch
fi = torch.load("/content/drive/MyDrive/labeled_benchmark/embeddings.pt")
fi[0]

[tensor([[[ 1.0432, -0.0083, -0.5479,  ...,  0.3866, -0.7958, -0.1065],
          [-0.0088,  0.1391,  0.5338,  ..., -0.1907,  0.6939,  0.2201],
          [ 0.1225,  0.0220,  0.6945,  ..., -0.0364,  0.4534,  0.2356],
          ...,
          [-0.0057,  0.0786,  0.5148,  ..., -0.0253,  0.3716,  0.1362],
          [-0.1135,  0.0271,  0.4351,  ...,  0.0197,  0.3706,  0.1608],
          [-0.1548, -0.0117,  0.4265,  ...,  0.0615,  0.3823,  0.1144]]]),
 tensor([[[ 1.0432, -0.0083, -0.5479,  ...,  0.3866, -0.7958, -0.1065],
          [-0.0088,  0.1391,  0.5338,  ..., -0.1907,  0.6939,  0.2201],
          [ 0.1225,  0.0220,  0.6945,  ..., -0.0364,  0.4534,  0.2356],
          ...,
          [-0.0057,  0.0786,  0.5148,  ..., -0.0253,  0.3716,  0.1362],
          [-0.1135,  0.0271,  0.4351,  ...,  0.0197,  0.3706,  0.1608],
          [-0.1548, -0.0117,  0.4265,  ...,  0.0615,  0.3823,  0.1144]]])]